In [19]:
import glob
from os.path import join, basename, splitext, abspath

In [20]:
def readFrameAnnotation(annotationFile):
    """
        read annotation file
        return the list of annotation ([start, end], gesture)
    """
    anno = []
    for l in open(annotationFile).read().splitlines():
        s = l.split(' ')
        anno += [ ([int(s[1]), int(s[2])], int(s[0])-1)]
    return anno

In [18]:
class Annotation:
    def __init__(self, annotFile=None):
        self.seqs = []
        self.gesture = []
        
        if not annotFile is None:
            for l in open(annotFile).read().splitlines():
                s = l.split(' ')
                self.seqs.append( (int(s[1]), int(s[2])) )
                self.gesture.append(int(s[0])-1)
    
    def addAnnotation(self, annotFile):
        for l in open(annotFile).read().splitlines():
            s = l.split(' ')
            self.seqs.append(int(s[1]), int(s[2]))
            self.gesture.append(int(s[0])-1)
            
    def __len__(self):
        return len(self.gesture)
    
    def __getitem__(self, i):
        return self.seqs[i], self.gesture[i]
    
    def __str__(self):
        s = "Annotation("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i])
        else :
            s += str(self[0]) + ' ... ' + str(self[-1])
        return s +")"
    
    def __repr__(self):
        s = "Annotation("
        for i in range(len(self)):
            s += str(self[i])
        return s +")"

In [17]:
class VideoDataset:
    def __init__(self, folderName):
        """
            Look for all .mp4 files in the folder
            The folder must have a annotation folder inside

            return a list of [ (video path, annotation) ]
        """
        p = join(folderName, "*.mp4")
        annoPath = join(folderName, "annotation")
        
        self.annotations = []
        self.videoList = []
        self.folderName = folderName
        
        for v in glob.glob(p):
            bname = basename(v)
            annoFile = join(annoPath, splitext(bname)[0])
            anno = Annotation(annoFile)
            self.annotations.append(anno)
            self.videoList.append(abspath(v)) #to avoid 2 identical files
            
    def __len__(self):
        return len(self.videoList)
    
    def __getitem__(self, i):
        return self.videoList[i], self.annotations[i]
    
    def __repr__(self):
        s = "VideoDataset("
        for i in range(len(self)):
            s += repr(self[i]) + '\n'
        return s + ')'
    
    def shuffle(self):
        """
            Inplace shuffle
        """
        from random import shuffle
        l = list(zip(self.videoList, self.annotations))
        shuffle(l)
        self.videoList, self.annotations = zip(*l)
    
    def __str__(self):
        s = "VideoDataset("
        if len(self) < 5 :
            for i in range(len(self)):
                s += str(self[i]) + '\n'
        else :
            s += str(self[0]) + '\n'*2 + str(self[1]) 
            s += '\n'*2 + ' ... ' + '\n'*2
            s += str(self[-1])
        return s + ')'
    
    def addFolder(self, folder):
        p = join(folderName, "*.mp4")
        for v in glob.glob(p):
            bname = basename(v)
            if not bname in self.videoList:
                annoFile = join(annoPath, splitext(bname)[0])
                annotation = Annotation(annoFile)
                bself.annotations.append(annotation)
                self.videoList.append(abspath(v)) #to avoid 2 identical files

In [100]:
if __name__=="__main__":
    d = VideoDataset("/video/Gesture")
    print(d)
    d.suffle()
    print(d)

VideoDataset(('/video/Gesture/fR01.mp4', Annotation(((0, 47), 5)((54, 89), 0)((93, 141), 5)((158, 194), 1)((200, 247), 5)((258, 302), 2)((307, 351), 5)((355, 393), 3)((398, 446), 5)((457, 503), 4)((508, 597), 5)((601, 647), 0)((650, 684), 5)((694, 733), 1)((736, 770), 5)((789, 827), 2)((830, 873), 5)((1015, 1056), 3)((1059, 1107), 5)((1112, 1156), 4)((1163, 1210), 5)))

('/video/Gesture/fR02.mp4', Annotation(((0, 19), 5)((26, 53), 0)((57, 102), 5)((113, 144), 1)((148, 191), 5)((199, 234), 2)((239, 287), 5)((291, 330), 3)((335, 374), 5)((381, 422), 4)((429, 496), 5)((501, 533), 0)((538, 577), 5)((581, 621), 1)((624, 667), 5)((671, 708), 2)((712, 748), 5)((753, 791), 3)((795, 834), 5)((839, 875), 4)((884, 918), 5)))

 ... 

('/video/Gesture/v42.mp4', Annotation(((0, 16), 5)((18, 40), 0)((46, 65), 5)((69, 90), 1)((99, 118), 5)((124, 144), 2)((151, 166), 5)((170, 197), 3)((200, 215), 5)((221, 241), 4)((252, 310), 5)((316, 333), 0)((338, 359), 5)((365, 386), 1)((393, 417), 5)((423, 443), 2)

In [28]:
def cutSequence(srcDir="/video/Gesture", destDir="/video/GestureSequence/"):
    """
        Cut all videos from srcDir into the destDir
        
        Each sequence will go in the subrepository corresponding to the class inside destDir
    """
    import cv2
    import os
    import os.path
    
    videoDataset = VideoDataset(srcDir)
    
    videoNumber = 0
    for video, annotation in videoDataset:
        vCap = cv2.VideoCapture(video)
        cframe = 0
        
        annoNumber = 0
        for (s0, s1), g in annotation:
            while cframe <= s0:
                cframe += 1
            
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            videoName = str(videoNumber)+"-"+str(annoNumber)+".avi"
            videoDir = os.path.join(destDir, str(g))
            if not os.path.isdir(videoDir):
                os.mkdir(videoDir)
            videoName = os.path.join(videoDir, videoName)
            record = cv2.VideoWriter(videoName, fourcc, 30, (int(vCap.get(3)), int(vCap.get(4))))
            
            while cframe <= s1:
                ret, frame = vCap.read()
                if ret:
                    record.write(frame)
                else:
                    return -1
                cframe +=1
            
            
            annoNumber +=1
        videoNumber += 1
    

In [ ]:
if __name__ == "__main__":
    cutSequence()

In [25]:
help(cv2.VideoWriter)

Help on built-in function VideoWriter:

VideoWriter(...)
    VideoWriter([filename, fourcc, fps, frameSize[, isColor]]) -> <VideoWriter object>

